# Problem 4 - Trajectory Optimization for a planar arm

## Start meshcat visualizer

Open the visualizer by clicking on the url in the output of this cell.

In [1]:
from pydrake.all import FindResourceOrThrow, StartMeshcat, MeshcatVisualizer
meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7001


## Compute the end effector (the ball) position and velocity using symbolic variables

This is how we get the expression in `kinematic_constraints.py`

In [2]:
# Computes the end effector velocity as xdot, ydot J_q (pos) * qdot

from sympy import *
import numpy as np

l, q0, q1, qdot0, qdot1 = symbols('l q0 q1 qdot0 qdot1', real=True)
pos = l * Matrix([-sin(q0), -cos(q0)]) + \
      l * Matrix([-sin(q0 + q1), -cos(q0 + q1)])
vel = Matrix(pos).jacobian([q0, q1]) @ np.reshape(np.array([qdot0, qdot1]), (2,1))

print('pos (x_ball, z_ball):\n', pos)
print('')
print('vel (x_ball dot, z_ball dot):\n', vel)

pos (x_ball, z_ball):
 Matrix([[-l*sin(q0) - l*sin(q0 + q1)], [-l*cos(q0) - l*cos(q0 + q1)]])

vel (x_ball dot, z_ball dot):
 Matrix([[-l*qdot1*cos(q0 + q1) + qdot0*(-l*cos(q0) - l*cos(q0 + q1))], [l*qdot1*sin(q0 + q1) + qdot0*(l*sin(q0) + l*sin(q0 + q1))]])


## Run trajectory optimization to find the optimal trajectory

In [3]:
import importlib
import catching_trajectory
importlib.reload(catching_trajectory)
from catching_trajectory import find_throwing_trajectory

N = 5
initial_state = np.zeros(14)
final_configuration = np.array([np.pi, 0])
tf = 3.0
distance = 15.0
%tb
x_traj, u_traj, prog, _, _ = find_throwing_trajectory(N, initial_state, final_configuration, distance, tf)


No traceback available to show.


n_q =  7
n_v =  7
n_x =  14
n_u =  7
initial state =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
optimal cost:  63.0625534578921
x_sol:  [[ 0.00000000e+00  9.26449609e-16  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.09965959e-01  3.41860875e-01  1.96354911e-01  2.14226169e-01
   7.71289831e-01 -8.03624507e-01 -1.24478657e-01 -2.18099971e-01
   1.60784162e+00  7.32122273e-01  1.29376312e+00  3.09677218e+00
   1.17689748e+00 -4.18431995e-01]
 [-9.06306487e-01  3.14159265e+00  8.64698124e-01 -7.85981733e-01
   3.14159265e+00  2.80672628e+00 -8.57156059e-01 -8.82475673e-01
   4.91674281e+00 -7.16215982e-01 -2.54439671e+00 -1.86238984e+00
   2.47794659e+00 -1.67194852e+00]
 [-6.82507752e-01  3.14155931e+00 -3.87298178e-01 -6.55767852e-01
  -3.14159265e+00 -1.78322465e-02 -3.14159265e+00  1.72450861e+00
  -5.73301792e+00 -2.55002544e+

## Visualize the optimal trajectory

This will animate the optimal trajectory in the meshcat window (see above)

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import time

from pydrake.all import (
    DiagramBuilder, Simulator, FindResourceOrThrow, MultibodyPlant, PiecewisePolynomial, SceneGraph,
    Parser, MultibodyPositionToGeometryPose, TrajectorySource, Demultiplexer, ConstantVectorSource
)


# Create a MultibodyPlant for the arm
file_name = "planar_arm.urdf"
builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
planar_arm = builder.AddSystem(MultibodyPlant(0.0))
planar_arm.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant=planar_arm).AddModels(file_name)
planar_arm.Finalize()

n_q = planar_arm.num_positions()
n_v = planar_arm.num_velocities()
n_u = planar_arm.num_actuators()


x_traj_source = builder.AddSystem(TrajectorySource(x_traj))
u_traj_source = builder.AddSystem(TrajectorySource(u_traj))

demux = builder.AddSystem(Demultiplexer(np.array([n_q, n_v])))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(planar_arm))
zero_inputs = builder.AddSystem(ConstantVectorSource(np.zeros(n_u)))

builder.Connect(zero_inputs.get_output_port(), planar_arm.get_actuation_input_port())
builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(planar_arm.get_source_id()))

MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
diagram.set_name("diagram")


# Visualize the motion for `n_playback` times
n_playback = 3
for i in range(n_playback):
  # Set up a simulator to run this diagram.
  simulator = Simulator(diagram)
  simulator.Initialize()
  simulator.set_target_realtime_rate(1)
  simulator.AdvanceTo(tf);
  time.sleep(2)

RuntimeError: /home/pradiptark/Documents/UPenn/Courses/1st_Semester/MEAM_5170_Control_and_Optimization/Final_Project/arm_final_project/planar_arm.urdf:0: error: Failed to parse XML file: XML_ERROR_FILE_NOT_FOUND

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a1190e5b-e06e-41db-a057-2e4bd9ffdb07' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>